In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np

In [5]:
bt = pd.read_csv("Bitcoin Historical Data (2014-2024).csv")
bt

,Date,Price,Open,High,Low,Vol.,Change %
0,03/24/2024,"67,211.9","64,036.5","67,587.8","63,812.9",65.59K,4.96%
1,03/23/2024,"64,037.8","63,785.6","65,972.4","63,074.9",35.11K,0.40%
2,03/22/2024,"63,785.5","65,501.5","66,633.3","62,328.3",72.43K,-2.62%
3,03/21/2024,"65,503.8","67,860.0","68,161.7","64,616.1",75.26K,-3.46%
4,03/20/2024,"67,854.0","62,046.8","68,029.5","60,850.9",133.53K,9.35%
...,...,...,...,...,...,...,...
3649,03/28/2014,482.6,460.5,515.0,453.8,4.11K,4.81%
3650,03/27/2014,460.5,562.5,567.8,460.5,3.78K,-18.13%
3651,03/26/2014,562.5,562.9,575.4,546.3,3.71K,-0.08%
3652,03/25/2014,562.9,567.6,569.7,550.4,3.87K,-0.82%


In [6]:
bt.dtypes

Date        object
Price       object
Open        object
High        object
Low         object
Vol.        object
Change %    object
dtype: object

In [7]:
#reformatting to drop k and m, change to integers

#function loop  to drop and multiply
def km_to_number(value):
    if isinstance(value, str):
        if 'K' in value:
            return float(value.replace('K', '')) * 1000
        elif 'M' in value:
            return float(value.replace('M', '')) * 1000000
        elif 'B' in value:
            return float(value.replace('B', '')) * 1000000000
    else:
        return value

bt["Vol."] = bt["Vol."].apply(km_to_number)

bt

,Date,Price,Open,High,Low,Vol.,Change %
0,03/24/2024,"67,211.9","64,036.5","67,587.8","63,812.9",65590.0,4.96%
1,03/23/2024,"64,037.8","63,785.6","65,972.4","63,074.9",35110.0,0.40%
2,03/22/2024,"63,785.5","65,501.5","66,633.3","62,328.3",72430.0,-2.62%
3,03/21/2024,"65,503.8","67,860.0","68,161.7","64,616.1",75260.0,-3.46%
4,03/20/2024,"67,854.0","62,046.8","68,029.5","60,850.9",133530.0,9.35%
...,...,...,...,...,...,...,...
3649,03/28/2014,482.6,460.5,515.0,453.8,4110.0,4.81%
3650,03/27/2014,460.5,562.5,567.8,460.5,3780.0,-18.13%
3651,03/26/2014,562.5,562.9,575.4,546.3,3710.0,-0.08%
3652,03/25/2014,562.9,567.6,569.7,550.4,3870.0,-0.82%


In [8]:
#convert to date to datetime, price/open/high/low to float, change% to float
bt["Date"] = pd.to_datetime(bt["Date"])
bt[["Price", "Open", "High", "Low"]] = bt[["Price", "Open", "High", "Low"]].applymap(lambda x: float(x.replace(',', '')))
bt['Change %'] = bt['Change %'].str.rstrip('%').astype(float)
bt

,Date,Price,Open,High,Low,Vol.,Change %
0,2024-03-24,67211.9,64036.5,67587.8,63812.9,65590.0,4.96
1,2024-03-23,64037.8,63785.6,65972.4,63074.9,35110.0,0.40
2,2024-03-22,63785.5,65501.5,66633.3,62328.3,72430.0,-2.62
3,2024-03-21,65503.8,67860.0,68161.7,64616.1,75260.0,-3.46
4,2024-03-20,67854.0,62046.8,68029.5,60850.9,133530.0,9.35
...,...,...,...,...,...,...,...
3649,2014-03-28,482.6,460.5,515.0,453.8,4110.0,4.81
3650,2014-03-27,460.5,562.5,567.8,460.5,3780.0,-18.13
3651,2014-03-26,562.5,562.9,575.4,546.3,3710.0,-0.08
3652,2014-03-25,562.9,567.6,569.7,550.4,3870.0,-0.82


In [9]:
bt.dtypes

Date        datetime64[ns]
Price              float64
Open               float64
High               float64
Low                float64
Vol.               float64
Change %           float64
dtype: object

In [10]:
bt['Date'] = bt['Date'].astype('int64')
bt['Date'] = bt['Date'].astype(float)

In [11]:
#adding a classification for softmax activation function
conditions = [
    (bt['Change %'] > 0),
    (bt['Change %'] < 0),
    (bt['Change %'] == 0)
]
values = [1, 0, 2]

bt['Inertia'] = np.where(conditions[0], values[0],
                np.where(conditions[1], values[1], values[2]))
bt

,Date,Price,Open,High,Low,Vol.,Change %,Inertia
0,1.711238e+18,67211.9,64036.5,67587.8,63812.9,65590.0,4.96,1
1,1.711152e+18,64037.8,63785.6,65972.4,63074.9,35110.0,0.40,1
2,1.711066e+18,63785.5,65501.5,66633.3,62328.3,72430.0,-2.62,0
3,1.710979e+18,65503.8,67860.0,68161.7,64616.1,75260.0,-3.46,0
4,1.710893e+18,67854.0,62046.8,68029.5,60850.9,133530.0,9.35,1
...,...,...,...,...,...,...,...,...
3649,1.395965e+18,482.6,460.5,515.0,453.8,4110.0,4.81,1
3650,1.395878e+18,460.5,562.5,567.8,460.5,3780.0,-18.13,0
3651,1.395792e+18,562.5,562.9,575.4,546.3,3710.0,-0.08,0
3652,1.395706e+18,562.9,567.6,569.7,550.4,3870.0,-0.82,0


In [12]:
bit = bt.copy()
bit.drop(columns=["Date"])

,Price,Open,High,Low,Vol.,Change %,Inertia
0,67211.9,64036.5,67587.8,63812.9,65590.0,4.96,1
1,64037.8,63785.6,65972.4,63074.9,35110.0,0.40,1
2,63785.5,65501.5,66633.3,62328.3,72430.0,-2.62,0
3,65503.8,67860.0,68161.7,64616.1,75260.0,-3.46,0
4,67854.0,62046.8,68029.5,60850.9,133530.0,9.35,1
...,...,...,...,...,...,...,...
3649,482.6,460.5,515.0,453.8,4110.0,4.81,1
3650,460.5,562.5,567.8,460.5,3780.0,-18.13,0
3651,562.5,562.9,575.4,546.3,3710.0,-0.08,0
3652,562.9,567.6,569.7,550.4,3870.0,-0.82,0


In [13]:

y = bit["Inertia"].values
X = bit.drop(columns="Inertia").values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30
num_classes = 3

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=num_classes, activation="softmax"))



c:\Users\lucas\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
#Compile
nn.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

#Fit
nn.fit(X_train_scaled, y_train, epochs=50, verbose=1)

Epoch 1/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6426 - loss: 0.8847
Epoch 2/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9417 - loss: 0.3116
Epoch 3/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9726 - loss: 0.1570
Epoch 4/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9801 - loss: 0.1145
Epoch 5/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9806 - loss: 0.0958
Epoch 6/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9863 - loss: 0.0678
Epoch 7/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9911 - loss: 0.0638
Epoch 8/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9859 - loss: 0.0621
Epoch 9/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9925 - loss: 0.0427
Epoch 10/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9936 - loss: 0.0366
Epoch 11/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9932 - loss: 0.0391
Epoch 12/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9912 - lo

In [16]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

29/29 - 0s - 11ms/step - accuracy: 0.9891 - loss: 0.0278
Loss: 0.02776368334889412, Accuracy: 0.9890590906143188
